In [1]:
import os
import pandas as pd
import zipfile
import csv
from io import StringIO
import mysql.connector
import re

### Testing on home server

In [ ]:
try:
    connection = mysql.connector.connect(
        host = '192.168.1.26',
        user = 'aportra',
        password = '@$Occer22',
        database = 'wedge_assignment')

    cursor = connection.cursor()
except mysql.connector.Error as err:
    print(f'{err}')

finally:
    if connection.is_connected():
        connection.close()
        print('Connection Closed')
    
    if cursor:
        print('Cursor created')

#### Setting Directory for Mac and Windows 

In [3]:
try:
    directory_small = '/Users/aaronportra/Documents/ADA/wedge_project/WedgeZipOfZips_small/'
    directory = '/Users/aaronportra/Documents/ADA/wedge_project/WedgeZipOfZips/'
    os.listdir(directory)
except FileNotFoundError:
    directory_small = 'C:\\Users\\aport\\OneDrive\\Documents\\School\\Fall Semester 2024\\Applied Data Analytics\\wedge_project\\WedgeZipOfZips_small\\'
    directory = 'C:\\Users\\aport\\OneDrive\\Documents\\School\\Fall Semester 2024\\Applied Data Analytics\\wedge_project\\WedgeZipOfZips\\'
    os.listdir(directory)

## Cleaning Data for BQ

In [5]:
def sniff(file):

    file = StringIO(file)

    sample = file.read(10000)

    sniffer = csv.Sniffer()

    file.seek(0)

    delimiter = sniffer.sniff(sample).delimiter


    reader = csv.reader(file, delimiter=delimiter)

    first_row = next(reader)

    has_header = sniffer.has_header(sample)


    if len(first_row) < 50:
        has_header = False

    return delimiter, has_header

In [329]:
# #Adding space to text. Example: ChickenBreast -> Chicken Breast
# def add_space(text):
#     if isinstance(text,str):
#         return re.sub(r'([a-z])([A-Z])',r'\1 \2',text)

#     return text


In [7]:
#Read in files and load to pandas

def create_db(directory):
    transactions = {}

    for file in os.listdir(directory):
        if file.endswith('.zip'):
            with zipfile.ZipFile(os.path.join(directory, file),'r') as zip_file:
                for info in zip_file.namelist():
        
                    file_content = zip_file.read(info).decode('utf-8',errors = 'replace')

                    csv_file = StringIO(file_content)

                    h = sniff(file_content)

                    csv_file.seek(0)

                    if h[1]:
                        data = pd.read_csv(csv_file, delimiter=h[0])
                        data = pd.DataFrame(data)

                        columns = data.columns


                    elif not h[1]:
                        data = pd.read_csv(csv_file, delimiter = h[0], header = None,)
                        data = pd.DataFrame(data)
                        data.columns = columns
    
                    # del_nan(data)

                    data = data.convert_dtypes()

                transactions[file.rstrip('.zip')] = data


 

    for trans in transactions: 

        transactions[trans]['datetime'] = pd.to_datetime(transactions[trans]['datetime'])
        transactions[trans]['description'] = transactions[trans]['description'].str.replace('"','')
        for col in transactions[trans]:

            if 'price' in col.lower() or 'percent' in col.lower():
                transactions[trans][col] = pd.to_numeric(transactions[trans][col],errors = 'coerce').astype('float64')
                # if 'percent' not in col.lower():
                #     transactions[trans][col] = transactions[trans][col].fillna(0.0)



    
    return transactions

In [ ]:
# small_zips = create_db(directory_small)

full_zips = create_db(directory)

In [31]:
#Saving to CSV

save_path = '/Users/aaronportra/Documents/ADA/wedge_project/clean_transactions/'

for trans in full_zips:
    full_zips[trans].to_csv(save_path + trans + '.csv', encoding = 'utf-8', index = False)

